## Data Ingestion

In [1]:
from collections import namedtuple
import os
import urllib.request as request
from zipfile import ZipFile

In [2]:
DataIngestionConfig= namedtuple("DataIngestionConfig", [
    "root_dir",
    "source_URL",
    "local_data_file",
    "unzip_dir"
])

#### Add Named tuple to config Manager class

In [3]:
from deepClassifier.constants import *
from deepClassifier.utils.common import read_yaml, create_directories

In [17]:
class ConfigurationManager:
    def __init__(self, config_filepath= CONFIG_FILE_PATH,
                    params_filepath= PARAMS_FILE_PATH
                    ):
        self.config = read_yaml(config_filepath)
        self.params = read_yaml(params_filepath)
        create_directories([self.config.artifacts_root])
    
    ## Data ingestion config
    def get_data_ingestion_config(self) -> DataIngestionConfig:
        config= self.config.data_ingestion
        ##Create root directory
        create_directories([config.root_dir])
        
        data_ingestion_config= DataIngestionConfig(
                root_dir = config.root_dir,
                source_URL = config.source_URL,
                local_data_file = config.local_data_file,
                unzip_dir = config.unzip_dir
        )
        return data_ingestion_config

#### Data ingestion Component

In [13]:
class DataIngestion:
    def __init__(self,config: DataIngestionConfig):
        self.config = config
    
    ##Download file
    def download_file(self):
        ##If file is existing in the path\
        if not os.path.exists(self.config.local_data_file):
            
            ##Download the file
            request.urlretrieve(url= self.config.source_URL,
             filename = self.config.local_data_file)
            
    ##Getting the updated list of files
    def _get_updated_list_of_files(self, list_of_files):
        ## image name ends with .jpg and in the file path either Cat of Dog should be present
        ## Which means we are considering files from Cat and Dog folder with .jpg extension
        return [f for f in list_of_files if f.endswith(".jpg") and ("Cat" in f or "Dog" in f)]

    ##Preprocessing of files
    def _preprocess(self, zf:ZipFile, f:str, working_dir:str):
        target_file_path = os.path.join(working_dir, f)
        ##If unzip version is not extracted then only extract the files to unzip_dir
        if not os.path.exists(target_file_path):
            zf.extract(f, working_dir)
        ## If file size is absent then remove
        if os.path.getsize(target_file_path) == 0:
            os.remove(target_file_path)

    ##Un-zip the data and clean
    def unzip_and_clean(self):
        with ZipFile(file = self.config.local_data_file, mode= "r") as zf:
            ## get the list of files
            list_of_files = zf.namelist()
            updated_list_of_files = self._get_updated_list_of_files(list_of_files)

            for f in updated_list_of_files:
                self._preprocess(zf, f, self.config.unzip_dir)



In [18]:
try:
    config= ConfigurationManager()
    data_ingestion_config= config.get_data_ingestion_config()
    data_ingestion = DataIngestion(config= data_ingestion_config)
    data_ingestion.download_file()
    data_ingestion.unzip_and_clean()
except Exception as e:
    raise e

[2022-11-16 12:09:23,511: INFO: common]: yaml file: configs/config.yaml loaded successfully
[2022-11-16 12:09:23,513: INFO: common]: yaml file: params.yaml loaded successfully
[2022-11-16 12:09:23,514: INFO: common]: created directory at: artifacts
[2022-11-16 12:09:23,515: INFO: common]: created directory at: artifacts/data_ingestion
DataIngestionConfig(root_dir='artifacts/data_ingestion', source_URL='https://download.microsoft.com/download/3/E/1/3E1C3F21-ECDB-4869-8368-6DEBA77B919F/kagglecatsanddogs_5340.zip', local_data_file='artifacts/data_ingestion/data.zip', unzip_dir='artifacts/data_ingestion')
